In [1]:
import numpy as np

In [57]:
def L1FeatureSign(A, gamma, y):
    pseudo = np.matmul(A.T, A)
    theta = np.zeros(pseudo.shape[0], dtype=np.int8)
    x = np.zeros(pseudo.shape[0])
    ATy = np.dot(A.T, y)
    gradient = -2 * ATy
    active_set = set()
    y2 = np.dot(y.T, y)
    
    flag1 = True
    flag2 = False
    
    while flag1 or flag2:
        if not flag2:
            xi = np.argmax(np.abs(gradient) * (theta==0))
            if gradient[xi] > gamma:
                theta[xi] = -1
                active_set.add(xi)
            elif gradient[xi] < -gamma:
                theta[xi] = 1
                active_set.add(xi)
            
        indices = np.array(sorted(active_set))
        pseudoHat = pseudo[np.ix_(indices, indices)]
        ATyHat = ATy[indices]
        thetaHat = theta[indices]
        b = ATyHat-(gamma*thetaHat)/2
        xHat = np.linalg.solve(np.atleast_2d(pseudoHat), b)
        thetaNew = np.sign(xHat)
        oldx = x[indices]
        signChanges = np.where(abs(thetaNew - thetaHat) > 1)[0]
        if len(signChanges) > 0:
            best_x = xHat
            Ax2 = np.dot(xHat, np.dot(ATyHat,xHat))
            ab = np.dot(xHat, ATyHat)
            obj = (y2 + Ax2- 2 * ab + gamma*abs(xHat).sum())
            for sign in signChanges:
                x = xHat[sign]
                y = oldx[sign]
                prop = y / (y - x)
                curr = oldx - prop * (oldx - xHat)
                temp1 = np.dot(curr, np.dot(pseudoHat, curr))
                temp2 = np.dot(curr, ATyHat)
                currentCost = y2 + (temp1-2*temp2+ gamma*abs(curr).sum())
                if currentCost < obj:
                    obj = currentCost
                    best_prop = prop
                    best_x = curr
        else:
            best_x = xHat
        x[indices] = best_x
        zeros = indices[np.abs(x[indices]) < 1e-18]
        x[zeros] = 0.
        theta[indices] = np.int8(np.sign(x[indices]))
        active_set.difference_update(zeros)
        gradient = - 2 * ATy + 2 * np.dot(pseudo, x)
        z = np.max(abs(gradient[theta == 0]))
        #z=np.inf
        nz = np.max(abs(gradient[theta != 0] + gamma * theta[theta != 0]))
        
        if z > gamma:
            flag1 = True
        else:
            flag1 = False
            
        if not np.allclose(nz,0):
            flag2 = True
        else:
            flag2 = False
    return x
          

In [58]:
A = np.array([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,1,0],[0,0,0,1,0,1]])

gamma = 0.3
y = np.array([4,6,2,3])

x = L1FeatureSign(A,gamma,y)
x


[1]
[0 1]
[0 1 3]
[0 1 2 3]


array([3.85, 5.85, 1.85, 2.85, 0.  , 0.  ])